In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data import morningstar

import numpy as np
import pandas as pd




In [ ]:
## Quantopian Help - Fundamentals https://www.quantopian.com/help/fundamentals#valuation
   
## enterprise_value : This number tells you what cash return you would get if you bought the entire company, including its debt. 
## ebit: Earnings minus expenses (excluding interest and tax expenses). 


class Value(CustomFactor):

    inputs = [morningstar.income_statement.ebit,
              morningstar.valuation.enterprise_value]
    window_length = 1
    
    def compute(self, today, assets, out, ebit, ev):
        out[:] = ebit[-1] / ev[-1]
        
        
class Quality(CustomFactor):
    
    # Pre-declare inputs and window_length
    inputs = [morningstar.operation_ratios.roe,]
    window_length = 1
    
    def compute(self, today, assets, out, roe):
        out[:] = roe[-1]
        
        
class AvgDailyDollarVolumeTraded(CustomFactor):
    inputs = [USEquityPricing.close, USEquityPricing.volume]
    
    def compute(self, today, assets, out, close_price, volume):
        out[:] = np.mean(close_price * volume, axis=0)

def make_pipeline():
    pipe = Pipeline()

    # Basic value and quality metrics.
    value = Value()
    pipe.add(value, "value")
    quality = Quality()
    pipe.add(quality, "quality")
    
    # We only want to trade relatively liquid stocks.
    # Build a filter that only passes stocks that have $10,000,000 average
    # daily dollar volume over the last 20 days.
    dollar_volume = AvgDailyDollarVolumeTraded(window_length=20)
    is_liquid = (dollar_volume > 10000000.0) #1e7
    
    # We also don't want to trade penny stocks, which we define as any stock with an
    # average price of less than $5.00 over the last 200 days.
    sma_200 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=200)
    not_a_penny_stock = (sma_200 > 5)
    
    # Before we do any other ranking, we want to throw away these assets.
    initial_screen = (is_liquid & not_a_penny_stock)

    # Construct and add a Factor representing the average rank of each asset by our 
    # value and quality metrics. 
    # By applying a mask to the rank computations, we remove any stocks that failed 
    # to meet our initial criteria **before** computing ranks.  This means that the 
    # stock with rank 10.0 is the 10th-lowest stock that passed `initial_screen`.
    ## へえええーーー．
    combined_rank = (
        value.rank(mask=initial_screen) + 
        quality.rank(mask=initial_screen)
    )
    pipe.add(combined_rank, 'combined_rank')

    # Build Filters representing the top and bottom 200 stocks by our combined ranking system.
    # We'll use these as our tradeable universe each day.
    longs = combined_rank.top(200)
    shorts = combined_rank.bottom(200)
    
    # The final output of our pipeline should only include 
    # the top/bottom 200 stocks by our criteria.
    pipe.set_screen(longs | shorts)
    
    pipe.add(longs, 'longs')
    pipe.add(shorts, 'shorts')
    
    return pipe



In [ ]:
pipe = make_pipeline()
results = run_pipeline(pipe, start_date="2017-11-1", end_date="2017-11-15")


In [ ]:
## 毎月第一営業日　Open　minutes=30
## leveraged_etf_list 以外

output = results.ix["2017-11-1"]
ranks = output["combined_rank"]
long_ranks = ranks[output['longs']]
short_ranks = ranks[output['shorts']]
long_weights = long_ranks / long_ranks.sum()
short_weights = short_ranks / short_ranks.sum()


    

In [ ]:
終焉output.sort_values(by="combined_rank").head()

In [ ]:
long_weights.head()

In [ ]:
## どうやって30日のシミュレーションをReserchでやる？
## (これはぱんだ問題)

results.groupby(results.index.get_level_values(0).month)

In [ ]:
long_weights
df_long = get_pricing(long_weights.index, start_date="2017-11-1", end_date="2017-11-30", 
                      fields="price", frequency="daily")
df_short = get_pricing(short_weights.index, start_date="2017-11-1", end_date="2017-11-30", 
                      fields="price", frequency="daily")

In [ ]:
(df_long.pct_change().sum()*long_weights).sum(), (df_short.pct_change().sum()*short_weights).sum()

In [ ]:
df_long.pct_change().sum().plot(kind="bar")

In [ ]:
df_short.pct_change().sum().plot(kind="bar")

In [ ]:
df_long.pct_change().sum().sort_values().head(10)

In [ ]:
idx = pd.IndexSlice
results.loc[idx[:,[symbols("KEM")]],:]
